Original data from the Colorado Secretary of State's webpage here: https://historicalelectiondata.coloradosos.gov/eng/ 

In [ ]:
import pandas as pd
import re
import glob
import json
from google.oauth2 import service_account
import os
import difflib
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from webdriver_manager.chrome import ChromeDriverManager

from tqdm import tqdm

In [ ]:
bq_project_id = os.environ.get('BQ_PROJECT_ID')
bq_expenditure_table = os.environ.get('BQ_EXPENDITURE_TABLE')
# Establish BigQuery credentials
bq_account_creds = json.loads(os.environ.get('BQ_ACCOUNT_CREDS'))
bq_credentials = service_account.Credentials.from_service_account_info(bq_account_creds, scopes=["https://www.googleapis.com/auth/cloud-platform"])

In [ ]:
# Define functions to clean our result data files.
# The result files include lines for things like total, remove those so we only have votes assigned to candidates.
def candidates_in(df):
    return(df[df['is_pseudocandidate'] == 0])

# The 'primary_party' field is NaN for General Electionsn which can cause problems, replace those values with 0.
def primary_party_fill_in(df):
    df['primary_party'] = df['primary_party'].fillna(0)
    return df

# The various elections are recorded by the full date they were held, this function creates a column that's just the year for ease of grouping.
def extract_election_year_in(df):
    df['election_year'] = pd.to_datetime(df['election_date']).dt.year
    return df

# The total results per candidate are broken out by county, this function aggregates those up to the candidate level.
def total_votes_in(df):
    df = pd.DataFrame(df.groupby(['election_year', 'election_type', 'office_name', 'primary_party', 'district_name', 'candidate_party', 'candidate_name'])['votes'].sum()).reset_index()
    return df

# This function provides a numerical ranking of the votes received by each candidate in each race so we can easily identify the winners.
def calc_vote_rank_in(df):
    df['votes_rank'] = df.sort_values(['election_year', 'election_type', 'office_name', 'primary_party', 'district_name', 'votes'], ascending=False).groupby(['election_year', 'election_type', 'primary_party', 'district_name']).cumcount() + 1
    return df

# This function fixes obvious formatting inconsistencies in candidate names to facilitate our matching them to their committees.
def clean_names_in(df):
    def alpha_only(candidate_name):
        return(re.sub(r'[^A-Za-z ]+', '', candidate_name).lower())
    
    def first_last_only(candidate_name):
        name_lst = candidate_name.split()
        if name_lst[-1] in ['jr', 'sr', 'ii', 'iii', 'iv']:
            name_str = name_lst[0] + ' ' + name_lst[-2]
        else:
            name_str = name_lst[0] + ' ' + name_lst[-1]
            
        return(name_str)
    
    df['candidate_name'] = df['candidate_name'].apply(alpha_only)
    df['candidate_name'] = df['candidate_name'].apply(first_last_only)

    return df

In [ ]:
# This function will load and clean each result.csv file in our directory and then append them all into one master frame
def load_co_election_results():
    files_lst = glob.glob(r'./results/*results.csv*')
    df = pd.DataFrame()
    lst = []

    for file in files_lst:
        df = pd.read_csv(file)
        df = (df.pipe(candidates_in)
                .pipe(primary_party_fill_in)
                .pipe(extract_election_year_in)
                .pipe(total_votes_in)
                .pipe(calc_vote_rank_in)
                .pipe(clean_names_in))

        lst.append(df)
    
    results_df = pd.concat(lst)
    results_df.reset_index(drop=True, inplace=True)

    return results_df


In [ ]:
co_results_df = load_co_election_results()

In [ ]:
co_results_df = co_results_df.sort_values(['candidate_name', 'election_year', 'election_type'])

In [ ]:
# I consider an incumbent any candidate who has won a General Election prior to their current race, regardless of district.
# This loops through every candidate-race and determines flags those candidates who had previously been elected.
for i in tqdm(co_results_df.index):
    _election_year_int = co_results_df.loc[i, 'election_year']
    _candidate_name_str = co_results_df.loc[i, 'candidate_name']

    _df = pd.DataFrame()
    _df = co_results_df[(co_results_df['candidate_name'] == _candidate_name_str)
                        & (co_results_df['election_year'] < _election_year_int)
                        & (co_results_df['votes_rank'] == 1)
                        & (co_results_df['election_type'] == 'General')]

    if len(_df) > 0:
        _incumbent_int = 1
    else:
        _incumbent_int = 0
    
    co_results_df.loc[i, 'incumbent'] = _incumbent_int

In [ ]:
bq_query_str = '''
WITH candidate_years AS (
  SELECT
    CO_ID,
    (CASE
      WHEN MOD(EXTRACT(YEAR FROM ExpenditureDate), 2) = 0 THEN EXTRACT(YEAR FROM ExpenditureDate)
      ELSE EXTRACT(YEAR FROM ExpenditureDate) + 1
    END) AS election_year,
    (CASE WHEN CandidateName = 'nan' THEN CommitteeName ELSE CandidateName END) AS candidate_name
  FROM ''' + bq_expenditure_table + '''
  WHERE Jurisdiction = 'STATEWIDE'
    AND CommitteeType LIKE ('Candidate%')
)

SELECT
  DISTINCT *
FROM candidate_years
'''

In [ ]:
committee_candidate_df = pd.read_gbq(bq_query_str, project_id=bq_project_id, location='us-west1', credentials=bq_credentials, progress_bar_type='tqdm')

In [ ]:
committee_candidate_df = committee_candidate_df.pipe(clean_names_in)

In [ ]:
for i in tqdm(co_results_df.index):
    _election_year_int = co_results_df.loc[i, 'election_year']
    _candidate_name_str = co_results_df.loc[i, 'candidate_name']

    _df = pd.DataFrame()
    _df = committee_candidate_df[committee_candidate_df['election_year'] == _election_year_int]

    try:
        _candidate_matched_name_str = difflib.get_close_matches(_candidate_name_str, _df['candidate_name'])[0]

        if difflib.SequenceMatcher(None, _candidate_name_str, _candidate_matched_name_str).ratio() > 0.85:
            _df = _df[_df['candidate_name'] == _candidate_matched_name_str]
            _df = _df.sort_values('CO_ID', ascending=False)

            _committee_id_int = int(_df.iloc[0, 0])
        else:
            _committee_id_int = 0

    except Exception as e:
        if ('out of range') in str(e):
            _committee_id_int = 0
        else:
            raise

    co_results_df.loc[i, 'CO_ID'] = _committee_id_int

In [ ]:
# Initiate the browser driver and head to the TRACER candidate search page
browser = webdriver.Chrome(ChromeDriverManager().install())
browser.get("http://tracer.sos.colorado.gov/PublicSite/SearchPages/CandidateSearch.aspx")
element = browser.find_element("id","_ctl0_LeftMenu_hlnkSearch")
webdriver.ActionChains(browser).move_to_element(element).click(element).perform()

In [ ]:
# Commence searching TRACER for each candidate's Committee ID
party_dict = {
    'Libertarian': '10', 
    'Democratic': '2', 
    'Natural Law Party': '-1', 
    'Unaffiliated': '12',
    'Republican': '5', 
    'Green': '9', 
    'American Constitution': '7', 
    'Colorado Reform': '-1',
    'Write-In': '-1',
    'Unity Party of Colorado': '8', 
    'Approval Voting': '20',
    'Non Partisan': '-1', 
    'Prohibition': '-1',
    'Unknown': '-1',
    'Colorado Taxpayers Party': '-1', 
    'Independent': '-1',
    'Unaffiliated/Republican': '-1',
    'Farmer-Labor': '-1', 
    'Not Registered': '-1',
    'Democrat / Republican': '-1'
}

office_dict = {
    'State Representative': 'Colorado House',
    'State Senate': 'Colorado Senate',
    'Attorney General': 'Attorney General',
    'State Treasurer': 'State Treasurer',
    'Treasurer': 'State Treasurer',
    'Secretary of State': 'Secretary of State',
    'Governor': 'Governor'
}

district_dict = {
    'State Representative': 'House District ',
    'State Senate': 'Senate District ',
}

_jurisdiction_str = '99'
for i in tqdm(co_results_df[(co_results_df['CO_ID'] == 0)].index):
    _candidate_last_stem_str = co_results_df.loc[i, 'candidate_name'].split(' ')[-1][:4]
    _candidate_party_str = party_dict.get(co_results_df.loc[i, 'candidate_party'])
    _election_year_int = co_results_df.loc[i, 'election_year']
    _election_year_str = str(_election_year_int)
    _office_str = office_dict.get(co_results_df.loc[i, 'office_name'])

    # Initiate a candidate search
    element = browser.find_element("id", "_ctl0_LeftMenu_hlnkSearch")
    webdriver.ActionChains(browser).move_to_element(element).click(element).perform()
    element = browser.find_element("id", "_ctl0_LeftMenu_hlnkSearchCandidates")
    webdriver.ActionChains(browser).move_to_element(element).click(element).perform()

    element = browser.find_element("id", "_ctl0_Content_rblLastNameSearchType_0")
    webdriver.ActionChains(browser).move_to_element(element).click(element).perform()

    # Set the party of the candidate we're seeking
    select = Select(browser.find_element("id", "_ctl0_Content_ddlParty_ucddlParty")) 
    select.select_by_value(_candidate_party_str)

    # Set the election year to the year we're missing the committee for if TRACER can handle it.
    if (_election_year_int >= 2010) & (_election_year_int  not in (2011, 2012, 2013)):
        select = Select(browser.find_element("id", "_ctl0_Content_lstElectionYear"))
        select.select_by_value(_election_year_str)
    
    # Set the jurisdiction of the candidate's race for which we're searching
    select = Select(browser.find_element("id", "_ctl0_Content_ddlJurisdiction"))
    select.select_by_value(_jurisdiction_str)

    # Set the office of the candidate's race for which we're searching
    select = Select(browser.find_element("id", "_ctl0_Content_ddlOffice"))
    select.select_by_visible_text(_office_str)

    # Set the district of the candidate's race for which we're searching
    if _office_str not in ['Governor', 'Secretary of State', 'State Treasurer', 'Attorney General']:
        _district_str = str(co_results_df.loc[i, 'district_name'])
        if len(_district_str) == 1:
            _district_str = '0' + _district_str

        _district_str = district_dict.get(co_results_df.loc[i, 'office_name']) + _district_str

        if _district_str in ['House District 14', 'House District 44']:
            _district_str = _district_str.upper()

        select = Select(browser.find_element("id", "_ctl0_Content_ddlDistrict"))
        select.select_by_visible_text(_district_str)

    # Search for candidates of all statuses
    select = Select(browser.find_element("xpath", "//select[@id='_ctl0_Content_ddlStatus']"))
    select.select_by_value("Select all statuses...")

    # Search for candidate by first and last
    find_serial = browser.find_element("id", "_ctl0_Content_txtLastName")
    find_serial.send_keys(_candidate_last_stem_str)
    find_serial.send_keys(Keys.RETURN)

    element = browser.find_element("id", "_ctl0_lblMasterError")
    if element.text == '':

        # If there are multiple entries select the first one, elsewise start gathering data
        if browser.title == 'TRACER - Candidate Search':
            element = browser.find_element("id", "_ctl0_Content_dgdSearchResults__ctl2_lnkCandidate")
            webdriver.ActionChains(browser).move_to_element(element).click(element).perform()
        
        # Some folks won't have committees yet so if that comes up just move on
        try:
            _co_id_str = browser.find_element("id", "_ctl0_Content_lblCommitteeID").text
        except:
            _co_id_str = np.nan

    else:
        _co_id_str = np.nan
    
    # Update the dataframe with the found committee ID
    co_results_df.loc[i, 'CO_ID'] = _co_id_str

    # Return to search screen to look for next candidate
    browser.get("http://tracer.sos.colorado.gov/PublicSite/SearchPages/CandidateSearch.aspx")

browser.close()
    

In [ ]:
results_schema_lst = [
    {'name': 'election_year', 'type': 'INT64', 'mode': 'REQUIRED'},
    {'name': 'election_type', 'type': 'STRING', 'mode': 'REQUIRED'},
    {'name': 'office_name', 'type': 'STRING', 'mode': 'REQUIRED'},
    {'name': 'primary_party', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'district_name', 'type': 'STRING', 'mode': 'REQUIRED'},
    {'name': 'candidate_party', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'candidate_name', 'type': 'STRING', 'mode': 'REQUIRED'},
    {'name': 'committee_id', 'type': 'INT64', 'mode': 'NULLABLE'},
    {'name': 'incumbent', 'type': 'BOOLEAN', 'mode': 'REQUIRED'},
    {'name': 'votes', 'type': 'INT64', 'mode': 'REQUIRED'},
    {'name': 'votes_rank', 'type': 'INT64', 'mode': 'REQUIRED'}
]

In [ ]:
panda_dtypes_dict = {
    'STRING': 'str',
    'INT64': 'Int64',
    'FLOAT64': 'float64',
    'DATETIME': 'datetime',
    'BOOLEAN': 'bool'
}

def set_dtypes(df, schema_lst):
    col_dtypes_dict = {x.get('name'): panda_dtypes_dict.get(x.get('type')) for x in schema_lst}
    
    for col in list(df):
        d_type = col_dtypes_dict.get(col)

        print(f"Setting {col} to {d_type}.")
        if d_type == 'Int64':
            df[col] = df[col].fillna(0)
            df[col] = df[col].astype(d_type)
            df[col] = df[col].replace(0, np.nan)
        else:
            df[col] = df[col].astype(d_type)
    
    return(df)

co_results_df = set_dtypes(co_results_df, results_schema_lst)

In [ ]:
co_results_df.info()

In [ ]:
co_results_df.to_gbq(destination_table='co_campaign_finance.results', project_id=bq_project_id, if_exists='replace', table_schema=results_schema_lst, credentials=bq_credentials)
